<https://github.com/PolymathicAI/xVal>


In [1]:
import os

os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.95"

In [2]:
import icecream
from icecream import ic

icecream.install()
ic_disable = False  # Global variable to disable ic
if ic_disable:
    ic.disable()
ic.configureOutput(includeContext=True, contextAbsPath=True)

In [3]:
import ast
import re
from flax.struct import dataclass
from datetime import datetime as dt
from torch.utils.data import DataLoader
import seaborn as sns
import matplotlib.pyplot as plt
import hephaestus as hp
import jax
import jax.numpy as jnp
import numpy as np
import optax
from flax.training import orbax_utils
import orbax.checkpoint
import orbax
import pandas as pd
from flax.training import train_state
from icecream import ic
from jax import random
from jax.tree_util import tree_flatten
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange
from hephaestus.models.time_series_decoder import SimpleDS

pd.options.mode.copy_on_write = True

In [4]:
from transformers import FlaxBertModel, BertTokenizerFast
import jax.numpy as jnp

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = FlaxBertModel.from_pretrained(model_name)
tokenizer = BertTokenizerFast.from_pretrained(model_name)

# Get the embeddings matrix
embeddings = model.params["embeddings"]["word_embeddings"]["embedding"]

# Now you can access specific embeddings like this:
# For example, to get embeddings for tokens 23, 293, and 993:
selected_embeddings = jnp.take(embeddings, jnp.array([23, 293, 993]), axis=0)

# If you want to get embeddings for specific words:
words = ["hello", "world", "example"]
tokens = tokenizer.convert_tokens_to_ids(words)
word_embeddings = jnp.take(embeddings, jnp.array(tokens), axis=0)
word_embeddings.shape

Some weights of FlaxBertModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: {('pooler', 'dense', 'bias'), ('pooler', 'dense', 'kernel')}
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(3, 768)

In [5]:
embeddings[jnp.array([23, 293, 993])].shape

(3, 768)

In [6]:
def line2df(line, idx):
    data_rows = []
    line = ast.literal_eval(line)
    for i, time_step in enumerate(line["data"]):
        row = {"time_step": i}
        # Add position data for each planet
        for j, position in enumerate(time_step):
            row[f"planet{j}_x"] = position[0]
            row[f"planet{j}_y"] = position[1]
        data_rows.append(row)

    df = pd.DataFrame(data_rows)
    description = line.pop("description")
    step_size = description.pop("stepsize")
    for k, v in description.items():
        for k_prop, v_prop in v.items():
            df[f"{k}_{k_prop}"] = v_prop
    df["time_step"] = df["time_step"] * step_size
    df.insert(0, "idx", idx)

    return df

In [7]:
files = os.listdir("data")
if "planets.parquet" not in files:
    with open("data/planets.data") as f:
        data = f.read().splitlines()

        dfs = []
        for idx, line in enumerate(tqdm(data)):
            dfs.append(line2df(line, idx))
        print("Concatenating dfs...")
        df = pd.concat(dfs)
    df.to_parquet("data/planets.parquet")
else:
    df = pd.read_parquet("data/planets.parquet")


# Combine total mass of all planets into one column `planet<n>_m`
mass_regex = re.compile(r"planet(\d+)_m")
mass_cols = [col for col in df.columns if mass_regex.match(col)]
df["total_mass"] = df[mass_cols].sum(axis=1)

# Introduce categorical columns for the number of planets choose non null columns with mass
df["n_planets"] = df[mass_cols].notnull().sum(axis=1).astype("category")
# Create category acceleration if the sum of plane/d_[x,y, z] is greater than 0
df["acceleration_x"] = df[
    [col for col in df.columns if "planet" in col and "_x" in col]
].sum(axis=1)
# Set acceleration_x to "increasing" if greater than 0 else "decreasing"
df["acceleration_x"] = df["acceleration_x"].apply(
    lambda x: "increasing" if x > 0 else "decreasing"
)
df["acceleration_y"] = df[
    [col for col in df.columns if "planet" in col and "_y" in col]
].sum(axis=1)
df["acceleration_y"] = df["acceleration_y"].apply(
    lambda x: "increasing" if x > 0 else "decreasing"
)


df.describe()

,idx,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,...,planet3_y,planet3_m,planet3_a,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e,total_mass
count,5.563957e+06,5.563957e+06,5.563957e+06,5.563957e+06,5.563957e+06,5.563957e+06,4.165044e+06,4.165044e+06,5.563957e+06,5.563957e+06,...,2.783627e+06,2.783627e+06,2.783627e+06,2.783627e+06,1.392864e+06,1.392864e+06,1.392864e+06,1.392864e+06,1.392864e+06,5.563957e+06
mean,6.248635e+04,9.748911e+00,-1.339198e-01,7.391138e-02,-1.340140e-01,7.291389e-02,-1.305344e-01,7.065633e-02,2.999306e+00,1.624756e+00,...,6.559150e-02,2.996303e+00,1.623874e+00,9.980576e-01,-1.276881e-01,6.519469e-02,3.002531e+00,1.625815e+00,1.001317e+00,1.049149e+01
std,3.607949e+04,5.993534e+00,1.228071e+00,1.213232e+00,1.227950e+00,1.212650e+00,1.217229e+00,1.203678e+00,1.157182e+00,5.876632e-01,...,1.200148e+00,1.153190e+00,5.270725e-01,5.764675e-01,1.211648e+00,1.199625e+00,1.156856e+00,5.167198e-01,5.779763e-01,3.991780e+00
min,0.000000e+00,0.000000e+00,-3.294763e+00,-2.997514e+00,-3.284004e+00,-2.998546e+00,-3.289790e+00,-2.998050e+00,1.000003e+00,1.000000e+00,...,-2.997621e+00,1.000054e+00,1.000000e+00,9.369537e-05,-3.273603e+00,-2.998913e+00,1.000103e+00,1.000000e+00,6.720938e-05,2.014597e+00
25%,3.124400e+04,4.655172e+00,-1.030131e+00,-9.020907e-01,-1.030516e+00,-9.028009e-01,-1.050662e+00,-9.211662e-01,1.993948e+00,1.000000e+00,...,-9.321272e-01,1.996853e+00,1.191548e+00,4.980967e-01,-1.071974e+00,-9.394428e-01,2.004240e+00,1.215927e+00,5.032645e-01,7.282371e+00
50%,6.249100e+04,9.523810e+00,-1.542335e-01,1.117099e-01,-1.538916e-01,1.099474e-01,-1.525520e-01,1.118031e-01,2.994477e+00,1.543047e+00,...,1.067355e-01,3.001879e+00,1.535683e+00,9.955040e-01,-1.507184e-01,1.055276e-01,3.000454e+00,1.519040e+00,1.003118e+00,1.028205e+01
75%,9.372800e+04,1.440000e+01,8.583344e-01,9.784987e-01,8.581045e-01,9.783998e-01,8.762358e-01,9.912902e-01,4.005747e+00,2.020672e+00,...,9.956064e-01,3.986406e+00,1.969644e+00,1.497746e+00,8.906780e-01,9.991220e-01,4.003332e+00,1.950518e+00,1.503457e+00,1.345633e+01
max,1.249990e+05,2.400000e+01,2.996370e+00,2.999014e+00,2.993319e+00,2.999536e+00,2.990464e+00,2.998478e+00,4.999994e+00,2.999984e+00,...,3.000881e+00,4.999990e+00,2.999909e+00,1.999957e+00,2.985180e+00,2.998936e+00,4.999679e+00,2.999497e+00,1.999999e+00,2.382455e+01


In [8]:
# Get train test split at 80/20
train_idx = int(df.idx.max() * 0.8)
train_df = df.loc[df.idx < train_idx].copy()
test_df = df.loc[df.idx >= train_idx].copy()
# del df
train_ds = SimpleDS(train_df)
test_ds = SimpleDS(test_df)
len(train_ds), len(test_ds)

(99999, 25001)

In [9]:
test_df

,idx,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,...,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e,total_mass,n_planets,acceleration_x,acceleration_y
0,0,0.000000,-0.274094,1.658928,-1.598680,1.237278,-0.072378,1.334127,3.092371,1.67039,...,0.265969,NaN,NaN,NaN,NaN,NaN,10.247159,4,decreasing,increasing
1,0,0.733333,-0.810119,1.516448,-1.860540,0.797326,-0.675005,1.164327,3.092371,1.67039,...,0.265969,NaN,NaN,NaN,NaN,NaN,10.247159,4,decreasing,increasing
2,0,1.466667,-1.261577,1.214381,-2.002381,0.305935,-1.131812,0.742120,3.092371,1.67039,...,0.265969,NaN,NaN,NaN,NaN,NaN,10.247159,4,decreasing,increasing
3,0,2.200000,-1.587840,0.791168,-2.015313,-0.205141,-1.347517,0.161522,3.092371,1.67039,...,0.265969,NaN,NaN,NaN,NaN,NaN,10.247159,4,decreasing,increasing
4,0,2.933333,-1.762252,0.291976,-1.898518,-0.702988,-1.278262,-0.453284,3.092371,1.67039,...,0.265969,NaN,NaN,NaN,NaN,NaN,10.247159,4,decreasing,decreasing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,25000,18.139535,0.859903,-0.357431,-1.780663,-0.841087,NaN,NaN,2.983244,1.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,7.554701,2,decreasing,decreasing
40,25000,18.604651,0.917215,0.133881,-1.609198,-1.111904,NaN,NaN,2.983244,1.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,7.554701,2,decreasing,decreasing
41,25000,19.069767,0.732278,0.590216,-1.391311,-1.350590,NaN,NaN,2.983244,1.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,7.554701,2,decreasing,decreasing
42,25000,19.534884,0.360838,0.898121,-1.132250,-1.549225,NaN,NaN,2.983244,1.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,7.554701,2,decreasing,decreasing


In [10]:
test_df.shape, train_df.shape

((1110975, 31), (4452982, 31))

In [11]:
df.idx.max()

124999

In [12]:
jnp.array([train_ds[0][0]])

Array([[[ 0.        ,  0.4651163 ,  0.9302326 , ...,         nan,
                 nan,         nan],
        [ 1.5600603 ,  1.689858  ,  1.7535888 , ...,         nan,
                 nan,         nan],
        [-0.854437  , -0.5143588 , -0.15420859, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [ 6.9741225 ,  6.9741225 ,  6.9741225 , ...,         nan,
                 nan,         nan]]], dtype=float32)

In [13]:
def make_batch(ds: SimpleDS, start: int, length: int):
    numeric = []
    categorical = []
    for i in range(start, length + start):
        numeric.append(ds[i][0])
        categorical.append(ds[i][1])
    # print index of None values
    return {"numeric": jnp.array(numeric), "categorical": jnp.array(categorical)}


batch = make_batch(train_ds, 0, 4)
# batch

In [14]:
# time_series_regressor = hp.simple_time_series.SimplePred(
#     train_ds, d_model=2048, n_heads=16 # large
# )
multiplier = 4
time_series_regressor = hp.time_series_decoder.SimplePred(
    train_ds, d_model=512, n_heads=8 * multiplier
)

In [15]:
# train_ds.reservoir_encoded

In [16]:
# train_ds.reservoir_encoded
train_ds.reservoir_encoded[jnp.array([0, 1, 2, 23])]

Array([[    0,     0,     0,     0,     0,     0,     0,     0],
       [ 1031, 16371, 25531,  1035,  7308,  1033,     0,     0],
       [  103,     0,     0,     0,     0,     0,     0,     0],
       [ 4774,  2509,  1035,  1049,     0,     0,     0,     0]],      dtype=int32)

In [17]:
batch["categorical"]

Array([[[33., 33., 33., 33., 33., 33., 35., 35., 35., 35., 35., 35.,
         35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35.,
         35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 33.,
         33., 33., 33., 33., 33., 33., 33., 33., nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [33., 33., 33., 33., 33., 33., 33., 33., 33., 33., 33., 33.,
         33., 33., 33., 33., 33., 33., 33., 33., 35., 35., 35., 35.,
         35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35.,
         35., 35., 35., 35., 35., 35., 35., 35., nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],

       [[35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35.,
         35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35.,
         35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 35.,
         35., 35., 35., 35., 35., 35., 35., 35., 35., 35., 33., 33.,
         33., 33., 33., 33., nan, nan, n

In [18]:
test_arr = jnp.array([1.0, 2.0, 3.0, 4.0])
# Convert to int
test_arr = test_arr.astype(jnp.int32)
test_arr

Array([1, 2, 3, 4], dtype=int32)

In [19]:
key = random.PRNGKey(0)
init_key, dropout_key = random.split(key)
vars = time_series_regressor.init(
    {"params": init_key, "dropout": dropout_key},
    batch["numeric"],
    categorical_inputs=batch["categorical"].astype(jnp.int32),
    deterministic=False,
)
dropout_key, original_dropout_key = random.split(dropout_key)

ic| time_series_decoder.py:294 in __call__()
    numeric_inputs.shape: (4, 27, 59)
ic| time_series_decoder.py:296 in __call__()- 'Here Again???'
ic| time_series_decoder.py:551 in __call__()
    "pe before tiling": 'pe before tiling'
    pe.shape: (1, 59, 512, 1)
ic| time_series_decoder.py:553 in __call__()
    "pe after tiling": 'pe after tiling'
    pe.shape: (4, 59, 512, 27)
ic| time_series_decoder.py:555 in __call__()
    "pe after transpose": 'pe after transpose'
    pe.shape: (4, 27, 59, 512)
ic| time_series_decoder.py:559 in __call__()
    "PE Result shape": 'PE Result shape'
    result.shape: (4, 27, 59, 512)
ic| time_series_decoder.py:354 in __call__()
    numeric_broadcast.shape: (4, 27, 59, 512)
    numeric_col_embeddings.shape: (4, 27, 59, 512)
ic| time_series_decoder.py:366 in __call__()
    "Masking for categorical data": 'Masking for categorical data'
ic| time_series_decoder.py:372 in __call__()
    mask_input.shape: (4, 29, 59)
ic| time_series_decoder.py:379 in __call__(

In [20]:
df.shape

(5563957, 31)

In [21]:
# ic.disable()

In [22]:
x = time_series_regressor.apply(
    vars,
    batch["numeric"],
    batch["categorical"].astype(jnp.int32),
    deterministic=False,
    rngs={"dropout": dropout_key},
)
print(x.get("numeric_out").shape)
# Check if categorical input is None and print None or it's shape
print(x.get("categorical_out").shape if x.get("categorical_out") is not None else None)

ic| time_series_decoder.py:294 in __call__()
    numeric_inputs.shape: (4, 27, 59)
ic| time_series_decoder.py:296 in __call__()- 'Here Again???'
ic| time_series_decoder.py:551 in __call__()
    "pe before tiling": 'pe before tiling'
    pe.shape: (1, 59, 512, 1)
ic| time_series_decoder.py:553 in __call__()
    "pe after tiling": 'pe after tiling'
    pe.shape: (4, 59, 512, 27)
ic| time_series_decoder.py:555 in __call__()
    "pe after transpose": 'pe after transpose'
    pe.shape: (4, 27, 59, 512)
ic| time_series_decoder.py:559 in __call__()
    "PE Result shape": 'PE Result shape'
    result.shape: (4, 27, 59, 512)
ic| time_series_decoder.py:354 in __call__()
    numeric_broadcast.shape: (4, 27, 59, 512)
    numeric_col_embeddings.shape: (4, 27, 59, 512)
ic| time_series_decoder.py:366 in __call__()
    "Masking for categorical data": 'Masking for categorical data'
ic| time_series_decoder.py:372 in __call__()
    mask_input.shape: (4, 29, 59)
ic| time_series_decoder.py:379 in __call__(

(4, 27, 59)
(4, 2, 59)


In [23]:
len(train_ds.categorical_indices)

2

In [24]:
# time_series_regressor.tabulate(
#     {"params": init_key, "dropout": dropout_key},
#     batch["numeric"],
#     console_kwargs={"force_jupyter": True, "width": 120},
# )

In [25]:
def calculate_memory_footprint(params):
    """Calculate total memory footprint of JAX model parameters and total
    number of parameters."""
    total_bytes = 0
    # Flatten the parameter tree structure into a list of arrays
    flat_params, _ = tree_flatten(params)
    for param in flat_params:
        # Calculate bytes: number of elements * size of each element
        bytes_per_param = param.size * param.dtype.itemsize
        total_bytes += bytes_per_param
    return total_bytes


def count_parameters(params):
    return sum(jnp.prod(jnp.array(p.shape)) for p in jax.tree_util.tree_leaves(params))


mem = calculate_memory_footprint(vars)
total_params = count_parameters(vars)


print(f"Memory of custom: {mem / 1e6:.2f} MB with {total_params:,} parameters")

Memory of custom: 202.17 MB with 50,542,877 parameters


In [26]:
mts_root_key = random.PRNGKey(44)
mts_main_key, ts_params_key, ts_data_key = random.split(mts_root_key, 3)

mask_data = False


def clip_gradients(gradients, max_norm):
    total_norm = jnp.sqrt(sum(jnp.sum(jnp.square(grad)) for grad in gradients.values()))
    scale = max_norm / (total_norm + 1e-6)
    clipped_gradients = jax.tree_map(
        lambda grad: jnp.where(total_norm > max_norm, grad * scale, grad), gradients
    )
    return clipped_gradients


def add_time_shifts(inputs: jnp.array, outputs: jnp.array) -> jnp.array:
    inputs_offset = 1
    inputs = inputs[:, :, inputs_offset:]
    tmp_null = jnp.full((inputs.shape[0], inputs.shape[1], inputs_offset), jnp.nan)
    inputs = jnp.concatenate([inputs, tmp_null], axis=2)
    nan_mask = jnp.isnan(inputs)
    inputs = jnp.where(nan_mask, jnp.zeros_like(inputs), inputs)
    print(f"{outputs.shape=}, {inputs.shape=}")
    outputs = jnp.where(nan_mask, jnp.zeros_like(outputs), outputs)

    return inputs, outputs, nan_mask


def numeric_loss(inputs, outputs):
    inputs, outputs, nan_mask = add_time_shifts(inputs, outputs)
    # TODO make loss SSL for values greater than 0.5 and MSE for values less than 0.5
    raw_loss = jnp.abs(outputs - inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss)
    loss = masked_loss.sum() / (~nan_mask).sum()
    return loss


def categorical_loss(inputs, outputs):
    inputs, outputs, nan_mask = add_time_shifts(inputs, outputs)

    raw_loss = optax.squared_error(outputs, inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss).mean()
    return masked_loss


def base_loss(
    numeric_inputs,
    categorical_inputs,
    outputs,
):
    numeric_out = outputs["numeric_out"]
    categorical_out = outputs["categorical_out"]
    print("Base Loss", numeric_inputs.shape, numeric_out.shape)
    numeric = numeric_loss(numeric_inputs, numeric_out)
    categorical = categorical_loss(categorical_inputs, categorical_out)
    return numeric + categorical


def base_loss_old(inputs, outputs):
    """TODO HERE IS THE SHIT"""
    # Remove the first value and add a jnp.nan to the end
    # inputs = inputs * 3
    inputs_offset = 1
    inputs = inputs[:, :, inputs_offset:]
    print(f"Inputs shape: {inputs.shape=}")
    # Add a jnp.nan to the end
    temp_null = jnp.full((inputs.shape[0], inputs.shape[1], inputs_offset), jnp.nan)
    inputs = jnp.concatenate([inputs, temp_null], axis=2)
    print(f"Inputs shape after addition: {inputs.shape=}")
    nan_mask = jnp.isnan(inputs)
    inputs = jnp.where(nan_mask, jnp.zeros_like(inputs), inputs)

    # outputs = outputs[:, :, :-inputs_offset]
    outputs = jnp.where(nan_mask, jnp.zeros_like(outputs), outputs)

    # raw_loss = optax.squared_error(outputs, inputs)
    # compute manually
    # raw_loss = jnp.square(outputs - inputs)
    # Abs loss
    raw_loss = jnp.abs(outputs - inputs)
    masked_loss = jnp.where(nan_mask, 0.0, raw_loss)
    loss = masked_loss.sum() / (~nan_mask).sum()

    return loss


def calculate_loss(
    params,
    state,
    numeric_inputs,
    categorical_inputs,
    dropout_key,
    mask_data: bool = True,
):
    outputs = state.apply_fn(
        {"params": params},
        # hp.mask_tensor(inputs, dataset, prng_key=mask_key),
        numeric_inputs=numeric_inputs,
        categorical_inputs=categorical_inputs.astype(jnp.int32),
        rngs={"dropout": dropout_key},
        deterministic=False,
        mask_data=mask_data,
    )
    loss = base_loss(
        numeric_inputs=numeric_inputs,
        categorical_inputs=categorical_inputs,
        outputs=outputs,
    )
    # Create mask for nan inputs

    return loss


@jax.jit
def train_step(
    state: train_state.TrainState,
    numeric_inputs,
    categorical_inputs,
    base_key,
    # mask_data=True,
):
    # print("In train step")
    dropout_key, mask_key, new_key = jax.random.split(
        base_key, 3
    )  # TODO Figure out mask key
    # print("Making masks")
    # numeric_inputs = batch["numeric"]
    # categorical_inputs = batch["categorical"]
    # print("Made masks")

    def calculate_loss_with_mask(params):
        return calculate_loss(
            params,
            state,
            numeric_inputs=numeric_inputs,
            categorical_inputs=categorical_inputs,
            dropout_key=dropout_key,
            mask_data=True,
        )

    def loss_fn(params):
        return calculate_loss_with_mask(params)

    # def calculate_loss_without_mask(params):
    #     return calculate_loss(params, state, batch, dropout_key, mask_data=False)

    # def loss_fn(params):
    #     return jax.lax.cond(
    #         mask_data,
    #         lambda _: calculate_loss_with_mask(params),
    #         lambda _: calculate_loss_without_mask(params),
    #         operand=None,
    #     )

    # def loss_fn(params):
    #     return calculate_loss(params, state, batch, dropout_key, mask_data=mask_data)

    grad_fn = jax.value_and_grad(loss_fn)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss, grad = grad_fn(state.params)
    # grad = replace_nans(grad)
    # grad = clip_gradients(grad, 1.0)
    state = state.apply_gradients(grads=grad)

    return state, loss, new_key


def evaluate(params, state, inputs, mask_data: bool = True):
    outputs = state.apply_fn(
        {"params": params},
        # hp.mask_tensor(inputs, dataset, prng_key=mask_key),
        inputs,
        deterministic=True,
        mask_data=mask_data,
    )
    loss = base_loss(inputs, outputs)
    return loss


@jax.jit
def eval_step(
    state: train_state.TrainState, numeric_inputs, categorical_inputs, base_key
):
    # mask_data=True
    mask_key, dropout_key, new_key = jax.random.split(base_key, 3)

    def calculate_loss_with_mask(params):
        return calculate_loss(
            params,
            state,
            numeric_inputs=numeric_inputs,
            categorical_inputs=categorical_inputs,
            dropout_key=dropout_key,
            mask_data=True,
        )

    def calculate_loss_without_mask(params):
        return calculate_loss(
            params, state=state, batch=batch, dropout_key=dropout_key, mask_data=False
        )

    def loss_fn(params):
        return calculate_loss_with_mask(params)

    # TODO Reimplement this...
    # def loss_fn(params):
    #     return jax.lax.cond(
    #         mask_data,
    #         lambda _: calculate_loss_with_mask(params),
    #         lambda _: calculate_loss_without_mask(params),
    #         operand=None,
    #     )

    # def loss_fn(params):
    #     return evaluate(params, state, batch, mask_data=mask_data)

    # (loss, individual_losses), grad = grad_fn(state.params)
    loss = loss_fn(state.params)
    return loss, new_key


def create_train_state(model, prng, batch, lr):
    init_key, dropout_key = random.split(prng)
    params = model.init(
        {"params": init_key, "dropout": dropout_key},
        batch["numeric"],
        batch["categorical"],
        deterministic=False,
    )
    # optimizer = optax.chain(optax.adam(lr))
    optimizer = optax.chain(optax.clip_by_global_norm(0.4), optax.adam(lr))
    # optimizer_state = optimizer.init(params)
    return train_state.TrainState.create(
        apply_fn=model.apply,
        params=params["params"],
        tx=optimizer,
        # tx_state=optimizer_state,
    )


batch_size = 2
# batch = train_ds[0]
# state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)
state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)

ic| time_series_decoder.py:294 in __call__()
    numeric_inputs.shape: (4, 27, 59)
ic| time_series_decoder.py:296 in __call__()- 'Here Again???'
ic| time_series_decoder.py:551 in __call__()
    "pe before tiling": 'pe before tiling'
    pe.shape: (1, 59, 512, 1)
ic| time_series_decoder.py:553 in __call__()
    "pe after tiling": 'pe after tiling'
    pe.shape: (4, 59, 512, 27)
ic| time_series_decoder.py:555 in __call__()
    "pe after transpose": 'pe after transpose'
    pe.shape: (4, 27, 59, 512)
ic| time_series_decoder.py:559 in __call__()
    "PE Result shape": 'PE Result shape'
    result.shape: (4, 27, 59, 512)
ic| time_series_decoder.py:354 in __call__()
    numeric_broadcast.shape: (4, 27, 59, 512)
    numeric_col_embeddings.shape: (4, 27, 59, 512)
ic| time_series_decoder.py:366 in __call__()
    "Masking for categorical data": 'Masking for categorical data'
ic| time_series_decoder.py:372 in __call__()
    mask_input.shape: (4, 29, 59)
ic| time_series_decoder.py:379 in __call__(

In [27]:
writer_name = "BERT_Embeddings"

writer_time = dt.now().strftime("%Y-%m-%dT%H:%M:%S")
model_name = writer_time + writer_name
train_summary_writer = SummaryWriter("runs/" + model_name)

MASK_DATA = True

test_set_key = random.PRNGKey(4454)

batch_size = 16
train_data_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

# train_data_loader = DataLoader(train_ds, batch_size=256 // 2, shuffle=True)
# test_data_loader = DataLoader(test_ds, batch_size=256 // 2, shuffle=True)

batch_count = 0
base_key = random.PRNGKey(42)

# Disable IC for training
max_iters = 200
ic.disable()
for j in trange(1, desc=f"epochs for {train_summary_writer.log_dir}"):
    # arrs = train_data_loader()
    for i in tqdm(train_data_loader, leave=False, desc="batches"):
        # for i in trange(len(pre_train) // batch_size, leave=False):
        # for i in trange(len(pre_train) // batch_size //10, leave=False):
        # batch = make_batch(train_ds, i[0], 4)
        state, loss, base_key = train_step(
            state,
            jnp.array(i[0]),
            jnp.array(i[1]),
            base_key,
            # mask_data=MASK_DATA,
        )
        if jnp.isnan(loss):
            raise ValueError("Nan Value in loss, stopping")
        batch_count += 1

        if batch_count % 1 == 0:
            train_summary_writer.add_scalar(
                "loss/loss", np.array(loss.item()), batch_count
            )
        if batch_count % 10 == 0:
            numeric_eval, categorical_eval = next(iter(test_data_loader))
            test_loss, base_key = eval_step(
                state,
                jnp.array(numeric_eval),
                jnp.array(categorical_eval),
                base_key,
                # mask_data=MASK_DATA,
            )
            train_summary_writer.add_scalar(
                "loss/test_loss", np.array(test_loss.item()), batch_count
            )
            train_summary_writer.flush()
        # if batch_count > 200:
        #     break
        if batch_count > max_iters:
            break

train_summary_writer.close()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


epochs for runs/2024-08-31T21:36:49BERT_Embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

batches:   0%|          | 0/6250 [00:00<?, ?it/s]

Base Loss (16, 27, 59) (16, 27, 59)
outputs.shape=(16, 27, 59), inputs.shape=(16, 27, 59)
outputs.shape=(16, 2, 59), inputs.shape=(16, 2, 59)
Base Loss (16, 27, 59) (16, 27, 59)
outputs.shape=(16, 27, 59), inputs.shape=(16, 27, 59)
outputs.shape=(16, 2, 59), inputs.shape=(16, 2, 59)


KeyboardInterrupt: 

In [57]:
train_ds[0][0]

array([[ 0.        ,  0.46511628,  0.93023256, ...,         nan,
                nan,         nan],
       [ 1.56006022,  1.68985799,  1.75358875, ...,         nan,
                nan,         nan],
       [-0.85443699, -0.5143588 , -0.15420858, ...,         nan,
                nan,         nan],
       ...,
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [ 6.97412249,  6.97412249,  6.97412249, ...,         nan,
                nan,         nan]])

In [60]:
state.apply_fn(
    {"params": state.params},
    # jnp.array(i[0]),
    # jnp.array(i[1]),
    jnp.array([train_ds[0][0]]),
    jnp.array([train_ds[0][1]]),
    deterministic=True,
    mask_data=False,
)

{'numeric_out': Array([[[11.181038  , 11.140583  , 11.101066  , ..., 11.32562   ,
          11.382234  , 11.41336   ],
         [ 0.22009788,  0.21450125,  0.20456655, ...,  0.15386526,
           0.16100045,  0.15643792],
         [-0.517749  , -0.5040334 , -0.49248892, ..., -0.59351164,
          -0.5940318 , -0.5888521 ],
         ...,
         [ 1.9436795 ,  1.9312918 ,  1.9221929 , ...,  1.9567974 ,
           1.9659095 ,  1.9736373 ],
         [ 0.02335496,  0.02150651,  0.02871914, ...,  0.08315714,
           0.08081253,  0.08272083],
         [11.387409  , 11.367141  , 11.331749  , ..., 11.304806  ,
          11.29361   , 11.277805  ]]], dtype=float32),
 'categorical_out': Array([[[35.3297  , 35.280807, 35.214005, 35.156345, 35.131405,
          35.14735 , 35.364372, 35.36472 , 35.332634, 35.293766,
          35.28131 , 35.294548, 35.324776, 35.352848, 35.359005,
          35.33747 , 35.311523, 35.280212, 35.24819 , 35.238476,
          35.266136, 35.264027, 35.25621 , 35.2493

In [ ]:
ckpt = {"model": state, "step": batch_count}


checkpoint_dir = f"checkpoints/{model_name}"
checkpoint_dir = os.path.abspath(checkpoint_dir)

# os.makedirs(checkpoint_dir, exist_ok=True)

orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
save_args = orbax_utils.save_args_from_target(ckpt)
orbax_checkpointer.save(checkpoint_dir, ckpt, save_args=save_args)

In [ ]:
new_checkpoint = orbax_checkpointer.restore(checkpoint_dir)
new_state = create_train_state(time_series_regressor, mts_main_key, batch, 0.0001)
new_state = new_state.replace(params=new_checkpoint["model"]["params"])

/Users/kailukowiak/Hephaestus/.venv/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1552: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


In [74]:
numeric_inputs, categorical_inputs = train_ds[0]
numeric_inputs = jnp.array([numeric_inputs])
categorical_inputs = jnp.array([categorical_inputs])
numeric_out, categorical_out = state.apply_fn(
    {"params": state.params},
    # jnp.array(i[0]),
    # jnp.array(i[1]),
    numeric_inputs,
    categorical_inputs,
    deterministic=True,
    mask_data=False,
)
numeric_out

'numeric_out'

In [76]:
@dataclass
class Results:
    numeric_out: jnp.array
    categorical_out: jnp.array
    numeric_inputs: jnp.array
    categorical_inputs: jnp.array


def return_results(state, dataset, idx=0, mask_start: int = None):
    numeric_inputs, categorical_inputs = dataset[idx]
    if mask_start:
        print("Masking")
        numeric_inputs = numeric_inputs[:, :mask_start]
        categorical_inputs = categorical_inputs[:, :mask_start]
    numeric_inputs = jnp.array([numeric_inputs])
    categorical_inputs = jnp.array([categorical_inputs])
    print("Getting results")
    out = state.apply_fn(
        {"params": state.params},
        # hp.mask_tensor(jnp.array([train_ds[0]]), dataset, prng_key=key),
        numeric_inputs=numeric_inputs,
        categorical_inputs=categorical_inputs,
        deterministic=True,
        mask_data=MASK_DATA,
    )
    numeric_out, categorical_out = out["numeric_out"], out["categorical_out"]
    return Results(numeric_out, categorical_out, numeric_inputs, categorical_inputs)


x = return_results(state, train_ds, 0)
x.categorical_out

Getting results


Array([[[36.12317 , 36.047028, 35.95711 , 35.88954 , 35.857315,
         35.866478, 36.01963 , 36.027714, 36.007008, 35.98618 ,
         35.985733, 36.00666 , 36.03651 , 36.057316, 36.04742 ,
         36.02263 , 35.992283, 35.962803, 35.935192, 35.915043,
         35.901955, 35.8892  , 35.87702 , 35.864693, 35.87282 ,
         35.886154, 35.89364 , 35.88587 , 35.881737, 35.879517,
         35.88059 , 35.89406 , 35.910313, 35.909245, 35.880554,
         35.68208 , 35.64167 , 35.620308, 35.61562 , 35.60777 ,
         35.59563 , 35.577515, 35.56537 , 35.577145, 35.555386,
         35.571976, 35.594948, 35.613667, 35.638573, 35.63228 ,
         35.61675 , 35.589813, 35.54513 , 35.525276, 35.54861 ,
         35.592075, 35.627785, 35.64456 , 35.621185],
        [39.11205 , 39.05336 , 38.982082, 38.928215, 38.907085,
         38.91555 , 38.94078 , 38.947823, 38.92335 , 38.898895,
         38.899143, 38.929794, 38.97196 , 39.00206 , 39.004402,
         38.99353 , 38.973038, 38.942234, 38.91171

In [77]:
mask_data = False
MASK_DATA = True


def process_results(arr: jnp.array, col_names: list):
    arr = jnp.squeeze(arr)
    df = pd.DataFrame(arr.T)
    df.columns = col_names
    return df


def show_results_df(state, base_df, dataset, idx: int = 0, mask_start: int = None):
    results = return_results(state, dataset, idx=idx, mask_start=mask_start)

    input_categorical = process_results(
        results.categorical_inputs, dataset.categorical_col_tokens
    )


process_results(x.numeric_out, train_ds.numeric_col_tokens)
# res = show_results_df(state, train_df, train_ds, idx=0, mask_start=10)

,time_step,planet0_x,planet0_y,planet1_x,planet1_y,planet2_x,planet2_y,planet0_m,planet0_a,planet0_e,...,planet3_y,planet3_m,planet3_a,planet3_e,planet4_x,planet4_y,planet4_m,planet4_a,planet4_e,total_mass
0,9.756296,0.111494,-0.117593,0.388406,-0.463551,-0.329284,0.918282,2.694641,1.768598,1.029188,...,0.161583,3.442658,1.673402,1.106791,-0.159440,0.221506,3.307631,1.792369,-0.087887,11.798042
1,9.642425,0.111467,-0.088864,0.424417,-0.436392,-0.369885,0.922729,2.668303,1.765564,1.019210,...,0.189700,3.484208,1.689074,1.099275,-0.155131,0.212693,3.311567,1.767895,-0.100261,11.788279
2,9.631383,0.094130,-0.077061,0.409449,-0.441680,-0.384776,0.933448,2.669973,1.756892,1.007583,...,0.203880,3.494935,1.709868,1.105471,-0.157640,0.217382,3.304543,1.767284,-0.090635,11.758847
3,9.642579,0.083382,-0.073815,0.389726,-0.447586,-0.399162,0.940623,2.676741,1.763083,1.002725,...,0.214067,3.492521,1.723492,1.111457,-0.156890,0.221923,3.306988,1.774342,-0.079743,11.731918
4,9.683665,0.078874,-0.077024,0.371988,-0.454990,-0.408621,0.939697,2.691235,1.775649,1.005562,...,0.220003,3.482205,1.726036,1.120321,-0.158326,0.219783,3.311317,1.787016,-0.068445,11.721029
5,9.756283,0.078238,-0.085264,0.359003,-0.463114,-0.403548,0.926282,2.705736,1.784645,1.016066,...,0.222133,3.471570,1.719362,1.134419,-0.161319,0.211063,3.312743,1.800762,-0.062613,11.731183
6,9.834229,0.039231,-0.096377,0.313503,-0.488958,-0.429806,0.896166,2.716848,1.766870,1.032279,...,0.234209,3.469087,1.736159,1.158088,-0.182317,0.201927,3.317343,1.810741,-0.046254,11.741310
7,9.912541,0.033579,-0.115894,0.308875,-0.498878,-0.415747,0.876612,2.723068,1.756351,1.044910,...,0.223370,3.470561,1.729179,1.164564,-0.192987,0.186054,3.322382,1.823186,-0.047983,11.749603
8,9.965851,0.025361,-0.127683,0.303464,-0.507985,-0.404321,0.864334,2.725231,1.746316,1.052034,...,0.215764,3.475831,1.730870,1.164768,-0.202951,0.174030,3.327143,1.833158,-0.051014,11.741024
9,10.004771,0.014708,-0.132921,0.295094,-0.518791,-0.397724,0.854481,2.724352,1.741161,1.053080,...,0.211820,3.483894,1.743762,1.163029,-0.210169,0.171736,3.330570,1.842648,-0.051750,11.726444


In [ ]:
def show_heatmap(df, title):
    """Shows heatmap for a dataframe
    excludes all columns that are only nan and all rows that are only nan"""

    df = df.dropna(axis=1, how="all")
    df = df.dropna(axis=0, how="all")
    plt.figure(figsize=(15, 10))
    cmap = sns.diverging_palette(220, 20, as_cmap=True)
    sns.heatmap(df, cmap=cmap, center=0, annot=True, fmt=".2f")
    plt.title(title)
    plt.show()


show_heatmap(res["diff_masked"], "Diff Masked")

NameError: name 'res' is not defined

In [ ]:
show_heatmap(res["diff_no_mask"].head(14), "Diff Masked")

In [ ]:
test_key = random.PRNGKey(4454)
x = jax.random.normal(test_key, (4, 26, 59, 256))

In [ ]:
no_mask_out = state.apply_fn(
    {"params": state.params},
    # jnp.array([test_ds[0][:10, :]]),
    jnp.array([test_ds[0][:, :10]]),
    deterministic=True,
    mask_data=True,
)
mask_out = state.apply_fn(
    {"params": state.params},
    jnp.array([test_ds[0][:, :20]]),
    deterministic=True,
    mask_data=True,
)
mask_out_df = pd.DataFrame(jnp.squeeze(mask_out).T)
mask_out_df.columns = test_df.columns[1:]
no_mask_out_df = pd.DataFrame(jnp.squeeze(no_mask_out).T)
no_mask_out_df.columns = test_df.columns[1:]

test_diff = mask_out_df - no_mask_out_df
test_diff

In [ ]:
def plot_planets(df_pred: pd.DataFrame, df_actual: pd.DataFrame, column: str, offset=0):
    plt.figure(figsize=(15, 10))
    plt.plot(df_pred[column], label="Autogregressive")
    plt.plot(df_actual[column], label="Actual")
    plt.title(f"{column} Predictions")
    plt.legend()
    # Show ticks and grid lines every 1 step
    plt.xticks(np.arange(0, len(df_pred), 1))
    plt.grid()
    # add black line at 0 on the y axis to show the difference
    plt.axhline(0, color="black")
    plt.show()

In [ ]:
def auto_regressive_predictions(
    state: train_state.TrainState, inputs: jnp.ndarray
) -> np.ndarray:
    # get the first row that contains all nan vales
    # if nan_rows_start >= stop_idx:
    #     return inputs
    nan_columns = jnp.isnan(inputs).all(axis=1)
    outputs = state.apply_fn(
        {"params": state.params},
        jnp.array([inputs]),
        # jnp.array([inputs]),
        deterministic=True,
        mask_data=MASK_DATA,
    )
    outputs = jnp.squeeze(outputs)
    final_row = np.array(outputs[:, -1])
    final_row = final_row[:, None]
    inputs = jnp.concatenate([inputs, final_row], axis=1)
    inputs = np.array(inputs)
    inputs[nan_columns] = np.nan
    return inputs
    # return auto_regressive_predictions(state, inputs, stop_idx)

In [ ]:
base_inputs = test_ds[300]
inputs_test = base_inputs[:, :10]
print(inputs_test.shape)
for i in trange(21):
    inputs_test = auto_regressive_predictions(state, inputs_test)

# x = auto_regressive_predictions(state, test_ds[0], 10)

In [ ]:
df_auto = pd.DataFrame(inputs_test.T)
df_actual = pd.DataFrame(base_inputs.T)
df_auto.columns = train_df.columns[1:]
df_actual.columns = train_df.columns[1:]
df_diff = df_auto - df_actual

# Drop rows that are all nan
df_diff = df_diff.dropna(axis=0, how="all")

In [ ]:
plot_planets(df_auto, df_actual, "time_step")

In [ ]:
plot_planets(df_auto, df_actual, "planet3_x")

In [ ]:
# inputs = jnp.array(next(iter(test_data_loader)))
inputs_test = make_batch(test_ds, 0, 1)

outputs = new_state.apply_fn(
    {"params": state.params},
    # hp.mask_tensor(inputs, dataset, prng_key=mask_key),
    inputs_test,
    deterministic=True,
    mask_data=True,
)
df_actual = pd.DataFrame(jnp.squeeze(inputs_test).T)
df_actual.columns = test_df.columns[1:]

df_pred = pd.DataFrame(jnp.squeeze(outputs).T)
df_pred.columns = test_df.columns[1:]
plot_planets(df_pred, df_actual, "time_step")
plot_planets(df_pred, df_actual, "planet2_y")

In [ ]:
inputs_test.shape

In [ ]:
show_heatmap(df_diff, "Auto Regressive Predictions")

In [ ]:
# plot planet0_x from df_auto and df_actual


res = show_results_df(state, train_df, train_ds, idx=299, mask_start=20)
plot_planets(res["pred"], res["actual_masked"], "planet2_y", offset=0)
# plot_planets(df_auto, df_actual, "planet2_y")

In [ ]:
# plot planet0_x from df_auto and df_actual
res = show_results_df(state, train_df, train_ds, idx=20, mask_start=20)
plot_planets(res["pred"], res["actual_no_mask"], "planet1_y", offset=0)

In [ ]:
res = show_results_df(state, train_df, test_ds, idx=0, mask_start=30)

plot_planets(res["pred"], res["actual_masked"], "planet2_x", offset=0)

In [ ]:
loss, key = eval_step(state, jnp.array(next(iter(test_data_loader))), base_key)
loss, key